In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from utils import mnist, plot_graphs, plot_mnist
import numpy as np

%matplotlib inline

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
train_loader, valid_loader, test_loader = mnist(valid=10000, transform=transforms.ToTensor())

In [5]:
class ConvLayer(nn.Module):
    def __init__(self, size, padding=1, pool_layer=nn.MaxPool2d(2, stride=2),
                 bn=False, dropout=False, activation_fn=nn.ReLU(), stride=1):
        super(ConvLayer, self).__init__()
        layers = []
        layers.append(nn.Conv2d(size[0], size[1], size[2], padding=padding, stride=stride))
        if pool_layer is not None:
            layers.append(pool_layer)
        if bn:
            layers.append(nn.BatchNorm2d(size[1]))
        if dropout:
            layers.append(nn.Dropout2d())
        layers.append(activation_fn)
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

In [6]:
class FullyConnected(nn.Module):
    def __init__(self, sizes, dropout=False, activation_fn=nn.Tanh):
        super(FullyConnected, self).__init__()
        layers = []
        
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if dropout:
                layers.append(nn.Dropout())
            layers.append(activation_fn())
        else: # нам не нужен дропаут и фнкция активации в последнем слое
            layers.append(nn.Linear(sizes[-2], sizes[-1]))
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

In [7]:
class Net(nn.Module):
    def __init__(self, batchnorm=False, dropout=False, optim_type='SGD', **optim_params):
        super(Net, self).__init__()
        
        self._conv1 = ConvLayer([1, 16, 4], padding=0, bn=batchnorm, stride=2, 
                                pool_layer=None, activation_fn=nn.Tanh())
        self._conv2 = ConvLayer([16, 32, 4], padding=0, bn=batchnorm, stride=2, 
                                pool_layer=None, activation_fn=nn.Tanh())
        self._conv3 = ConvLayer([32, 32, 3], padding=0, bn=batchnorm, stride=2, 
                                pool_layer=None, activation_fn=nn.Tanh())
            
        self.fc1 = FullyConnected([32*2*2, 32])
        self.fc2 = FullyConnected([32, 32])
        self.fc3 = FullyConnected([32, 32])
        self.fc4 = FullyConnected([32, 10])
        
        self._loss = None
        if optim_type == 'SGD':
            self.optim = optim.SGD(self.parameters(), **optim_params)
        elif optim_type == 'Adadelta':
            self.optim = optim.Adadelta(self.parameters(), **optim_params)
        elif optim_type == 'RMSProp':
            self.optim = optim.RMSprop(self.parameters(), **optim_params)
        elif optim_type == 'Adam':
            self.optim = optim.Adam(self.parameters(), **optim_params)
        
    def conv(self, x):
        l1 = self._conv1(x)
        l2 = self._conv2(l1)
        l3 = self._conv3(l2)
        return l3, l2, l1
        
    def forward(self, x):
        l3 = self.conv(x)[0]
        flatten = l3.view(-1, 32*2*2)
        x = self.fc1(flatten)
        x = self.fc2(x)
        x = self.fc3(x)
        h = self.fc4(x)
        return h
    
    def loss(self, output, target):
        self._loss = F.cross_entropy(output, target)
        return self._loss

In [33]:
device = torch.device("cpu") #"cuda:0" if torch.cuda.is_available() else 
print(device)
lr = 0.0001
prior_size = 10

class FC(nn.Module):
    def __init__(self, sizes, dropout=False, activation_fn=nn.Tanh(), flatten=False, 
                 last_fn=None, first_fn=None, device='cpu'):
        super(FC, self).__init__()
        layers = []
        self.flatten = flatten
        if first_fn is not None:
            layers.append(first_fn)
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if dropout:
                layers.append(nn.Dropout(dropout))
            layers.append(activation_fn) # нам не нужен дропаут и фнкция активации в последнем слое
        else: 
            layers.append(nn.Linear(sizes[-2], sizes[-1]))
        if last_fn is not None:
            layers.append(last_fn)
        self.model = nn.Sequential(*layers)
        self.to(device)
        
    def forward(self, x, y=None):
        if self.flatten:
            x = x.view(x.shape[0], -1)
        if y is not None:
            x = torch.cat([x, y], dim=1)
        return self.model(x)

caae_Enc = FC([28*28, 1024, 1024, prior_size], activation_fn=nn.LeakyReLU(0.2), flatten=True, device=device)
caae_Enc.load_state_dict(torch.load("./model_Enc_caae"))
caae_Enc.eval()

aaec_Enc = FC([28*28, 1024, 1024, prior_size], activation_fn=nn.LeakyReLU(0.2), flatten=True, device=device)
aaec_Enc.load_state_dict(torch.load("./model_Enc_aaec"))
aaec_Enc.eval()

cpu


FC(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=1024, out_features=10, bias=True)
  )
)

In [39]:
class CAAE_Clf(nn.Module):
    def __init__(self, conditional=True, batchnorm=False, dropout=False, optim_type='SGD', **optim_params):
        super(CAAE_Clf, self).__init__()
        self.conditional = conditional
        self.fc1 = FullyConnected([10, 32])
        self.fc2 = FullyConnected([32, 32])
        self.fc3 = FullyConnected([32, 64])
        self.fc4 = FullyConnected([64, 10])
        
        self._loss = None
        if optim_type == 'SGD':
            self.optim = optim.SGD(self.parameters(), **optim_params)
        elif optim_type == 'Adadelta':
            self.optim = optim.Adadelta(self.parameters(), **optim_params)
        elif optim_type == 'RMSProp':
            self.optim = optim.RMSprop(self.parameters(), **optim_params)
        elif optim_type == 'Adam':
            self.optim = optim.Adam(self.parameters(), **optim_params)
        
    def forward(self, x):
        if self.conditional:
            x = self.fc1(caae_Enc(x))
        else:
            x = self.fc1(aaec_Enc(x))

        x = self.fc2(x)
        x = self.fc3(x)
        h = self.fc4(x)
        return h
    
    def loss(self, output, target):
        self._loss = F.cross_entropy(output, target)
        return self._loss

In [40]:
models = {'Clf_Img': Net(False, False, optim_type='Adam', lr=1e-4), 
          'CAAE_clf': CAAE_Clf(True, False, False, optim_type='Adam', lr=1e-4, ),
          'AAEC_clf': CAAE_Clf(False, False, False, optim_type='Adam', lr=1e-4),
         }
train_log = {k: [] for k in models}
test_log = {k: [] for k in models}

In [41]:
def train(epoch, models, log=None):
    train_size = len(train_loader.sampler)
    for batch_idx, (data, target) in enumerate(train_loader):
        for model in models.values():
            model.optim.zero_grad()
            output = model(data)
            loss = model.loss(output, target)
            loss.backward()
            model.optim.step()
            
        if batch_idx % 200 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(train_loader))
            losses = ' '.join(['{}: {:.4f}'.format(k, m._loss.item()) for k, m in models.items()])
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(train_loader))
        losses = ' '.join(['{}: {:.4f}'.format(k, m._loss.item()) for k, m in models.items()])
        if log is not None:
            for k in models:
                log[k].append((models[k]._loss,))
        print(line + losses)

In [42]:
def test(models, loader, log=None):
    test_size = len(loader)
    test_loss = {k: 0. for k in models}
    with torch.no_grad():
        for data, target in loader:
            output = {k: m(data) for k, m in models.items()}
            for k, m in models.items():
                test_loss[k] += m.loss(output[k], target).item() # sum up batch loss
    
    for k in models:
        test_loss[k] /= test_size
    report = 'Test losses: ' + ' '.join(['{}: {:.4f}'.format(k, test_loss[k]) for k in test_loss])
    if log is not None:
        for k in models:
            log[k].append((test_loss[k],))
    print(report)

In [43]:
for epoch in range(1, 31):
    for model in models.values():
        model.train()
    train(epoch, models, train_log)
    for model in models.values():
        model.eval()
    test(models, valid_loader, test_log)

Train Epoch: 1 [0/50000 (0%)]	Losses Clf_Img: 2.3247 CAAE_clf: 2.3209 AAEC_clf: 2.2857
Train Epoch: 1 [10000/50000 (20%)]	Losses Clf_Img: 1.6804 CAAE_clf: 2.2199 AAEC_clf: 2.2630
Train Epoch: 1 [20000/50000 (40%)]	Losses Clf_Img: 0.7444 CAAE_clf: 2.1721 AAEC_clf: 2.0943
Train Epoch: 1 [30000/50000 (60%)]	Losses Clf_Img: 0.4252 CAAE_clf: 2.1169 AAEC_clf: 1.8069
Train Epoch: 1 [40000/50000 (80%)]	Losses Clf_Img: 0.3139 CAAE_clf: 2.0108 AAEC_clf: 1.6644
Train Epoch: 1 [50000/50000 (100%)]	Losses Clf_Img: 0.6897 CAAE_clf: 2.1100 AAEC_clf: 1.3404
Test losses: Clf_Img: 0.4613 CAAE_clf: 1.9714 AAEC_clf: 1.3775
Train Epoch: 2 [0/50000 (0%)]	Losses Clf_Img: 0.5623 CAAE_clf: 1.9688 AAEC_clf: 1.3076
Train Epoch: 2 [10000/50000 (20%)]	Losses Clf_Img: 0.3729 CAAE_clf: 1.9926 AAEC_clf: 1.1743
Train Epoch: 2 [20000/50000 (40%)]	Losses Clf_Img: 0.1627 CAAE_clf: 1.6799 AAEC_clf: 0.8553
Train Epoch: 2 [30000/50000 (60%)]	Losses Clf_Img: 0.3201 CAAE_clf: 1.7616 AAEC_clf: 0.9209
Train Epoch: 2 [40000/5000

Train Epoch: 14 [30000/50000 (60%)]	Losses Clf_Img: 0.0771 CAAE_clf: 1.7673 AAEC_clf: 0.9895
Train Epoch: 14 [40000/50000 (80%)]	Losses Clf_Img: 0.0821 CAAE_clf: 1.7694 AAEC_clf: 0.8242
Train Epoch: 14 [50000/50000 (100%)]	Losses Clf_Img: 0.0173 CAAE_clf: 1.5731 AAEC_clf: 0.8099
Test losses: Clf_Img: 0.0779 CAAE_clf: 1.8093 AAEC_clf: 0.8970
Train Epoch: 15 [0/50000 (0%)]	Losses Clf_Img: 0.0365 CAAE_clf: 1.7673 AAEC_clf: 0.7190
Train Epoch: 15 [10000/50000 (20%)]	Losses Clf_Img: 0.0325 CAAE_clf: 1.9285 AAEC_clf: 0.9229
Train Epoch: 15 [20000/50000 (40%)]	Losses Clf_Img: 0.0304 CAAE_clf: 1.7003 AAEC_clf: 0.9028
Train Epoch: 15 [30000/50000 (60%)]	Losses Clf_Img: 0.0108 CAAE_clf: 1.6557 AAEC_clf: 0.6148
Train Epoch: 15 [40000/50000 (80%)]	Losses Clf_Img: 0.0079 CAAE_clf: 1.6831 AAEC_clf: 0.8301
Train Epoch: 15 [50000/50000 (100%)]	Losses Clf_Img: 0.0182 CAAE_clf: 1.7086 AAEC_clf: 0.8852
Test losses: Clf_Img: 0.0789 CAAE_clf: 1.8091 AAEC_clf: 0.9002
Train Epoch: 16 [0/50000 (0%)]	Losses Cl

Train Epoch: 27 [50000/50000 (100%)]	Losses Clf_Img: 0.0129 CAAE_clf: 1.6709 AAEC_clf: 0.7357
Test losses: Clf_Img: 0.0666 CAAE_clf: 1.8057 AAEC_clf: 0.8969
Train Epoch: 28 [0/50000 (0%)]	Losses Clf_Img: 0.0049 CAAE_clf: 1.5469 AAEC_clf: 0.8301
Train Epoch: 28 [10000/50000 (20%)]	Losses Clf_Img: 0.0017 CAAE_clf: 1.8434 AAEC_clf: 1.2472
Train Epoch: 28 [20000/50000 (40%)]	Losses Clf_Img: 0.0057 CAAE_clf: 2.1220 AAEC_clf: 1.1655
Train Epoch: 28 [30000/50000 (60%)]	Losses Clf_Img: 0.0061 CAAE_clf: 1.7388 AAEC_clf: 1.1683
Train Epoch: 28 [40000/50000 (80%)]	Losses Clf_Img: 0.0583 CAAE_clf: 1.6886 AAEC_clf: 1.0106
Train Epoch: 28 [50000/50000 (100%)]	Losses Clf_Img: 0.0063 CAAE_clf: 1.8077 AAEC_clf: 0.8363
Test losses: Clf_Img: 0.0659 CAAE_clf: 1.8065 AAEC_clf: 0.8972
Train Epoch: 29 [0/50000 (0%)]	Losses Clf_Img: 0.0348 CAAE_clf: 1.7223 AAEC_clf: 0.9397
Train Epoch: 29 [10000/50000 (20%)]	Losses Clf_Img: 0.0214 CAAE_clf: 1.7845 AAEC_clf: 0.9464
Train Epoch: 29 [20000/50000 (40%)]	Losses Cl